In [30]:
# Real life data

import logging
import threading
import itertools
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import ibm_db

from pandas_profiling import ProfileReport
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
import seaborn as seabornInstance
from sqlalchemy import Column, Integer, String, Float, DateTime, Boolean, func
from iotfunctions import base
from iotfunctions import bif
from iotfunctions import entity
from iotfunctions import metadata
from iotfunctions.metadata import EntityType
from iotfunctions.enginelog import EngineLogging
from iotfunctions import estimator
from iotfunctions.ui import (UISingle, UIMultiItem, UIFunctionOutSingle,
                 UISingleItem, UIFunctionOutMulti, UIMulti, UIExpression,
                 UIText, UIStatusFlag, UIParameters)
from iotfunctions.dbtables import FileModelStore, DBModelStore
from mmfunctions.anomaly import (SaliencybasedGeneralizedAnomalyScore, SpectralAnomalyScore,
                 FFTbasedGeneralizedAnomalyScore, KMeansAnomalyScore, GBMRegressor)
import datetime as dt
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import MinCovDet
from sklearn import metrics
import scipy as sp
import scipy.fftpack
import skimage as ski  
from skimage import util as skiutil # for nifty windowing
import pyod as pyod
from pyod.utils.data import generate_data
from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize
from pyod.models.knn import KNN
from pyod.models.iforest import IForest
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

EngineLogging.configure_console_logging(logging.INFO)

In [54]:
credentials={"_id":"2d6dbddcd0ef52f54ab7e7a81f4face6","tenantId":"Monitor-Demo","db2":{"username":"bluadmin","password":"MjZkZTEwN2FjMWY1","databaseName":"BLUDB","port":50000,"httpsUrl":"https://dashdb-enterprise-yp-dal12-125.services.dal.bluemix.net:8443","host":"dashdb-enterprise-yp-dal12-125.services.dal.bluemix.net"},"iotp":{"url":"https://vrvzh6.internetofthings.ibmcloud.com/api/v0002","orgId":"vrvzh6","host":"vrvzh6.messaging.internetofthings.ibmcloud.com","port":8883,"asHost":"api-beta.connectedproducts.internetofthings.ibmcloud.com","apiKey":"a-vrvzh6-am4fwylysp","apiToken":"F+PF@HQhe_N*ecS4gb"},"messageHub":{"brokers":["kafka03-prod02.messagehub.services.us-south.bluemix.net:9093","kafka01-prod02.messagehub.services.us-south.bluemix.net:9093","kafka05-prod02.messagehub.services.us-south.bluemix.net:9093","kafka02-prod02.messagehub.services.us-south.bluemix.net:9093","kafka04-prod02.messagehub.services.us-south.bluemix.net:9093"],"username":"S0Zzzp1zZsF4hotd","password":"S1wyAP2jy9R2cmqcZwSfoqL5gByWSqn9"},"objectStorage":{"region":"global","username":"5c9b5139a83d4f68bd2fe458a2117fac","password":"5b282a68b84d070bd8674da4771998c6cb0743a4a0288129"},"config":{"objectStorageEndpoint":"https://undefined","bos_logs_bucket":"analytics-logs-monitor-demo-9c6aaaf268a1","bos_runtime_bucket":"analytics-runtime-monitor-demo-395729cb6a06","mh_topic_analytics_alerts":"analytics-alerts-Monitor-Demo"}}
db_schema=None
fm = FileModelStore()
db = Database(credentials=credentials, model_store=fm)
print (db)

2020-03-12T16:45:14.254 INFO iotfunctions.dbtables.__init__ Init FileModelStore
2020-03-12T16:45:14.255 INFO iotfunctions.db.__init__ Connection string for SqlAlchemy => db2): db2+ibm_db://bluadmin:MjZkZTEwN2FjMWY1@dashdb-enterprise-yp-dal12-125.services.dal.bluemix.net:50000/BLUDB;
2020-03-12T16:45:14.256 DEBUG iotfunctions.db.__init__ created a CosClient object
2020-03-12T16:45:16.239 DEBUG iotfunctions.db.__init__ Db connection established
2020-03-12T16:45:16.239 DEBUG urllib3.connectionpool._new_conn Starting new HTTPS connection (1): api-beta.connectedproducts.internetofthings.ibmcloud.com
2020-03-12T16:45:25.761 DEBUG urllib3.connectionpool._make_request https://api-beta.connectedproducts.internetofthings.ibmcloud.com:443 "GET /api/meta/v1/Monitor-Demo/entityType HTTP/1.1" 200 None
2020-03-12T16:45:26.589 DEBUG iotfunctions.db.http_request http request successful. status 200


In [34]:
#con = db.connection.connect()
DB2ConnString = 'DATABASE=' + credentials['db2']['databaseName'] + \
                ';HOSTNAME=' + credentials['db2']['host'] + \
                ';PORT=' + str(credentials['db2']['port']) + \
                ';PROTOCOL=TCPIP;UID=' + credentials['db2']['username'] + \
                ';PWD=' + credentials['db2']['password']
            
db_connection = ibm_db.connect(DB2ConnString, '', '')


# db.model = DBModelStore(credentials['tenantId'], None, credentials['db2']['username'], db.connection.connect(), 'db2'  )

In [46]:
model_store = DBModelStore(credentials['tenantId'], "TESTENTITYID", credentials['db2']['username'], db_connection, 'db2')
db.model_store = model_store

2020-03-12T16:37:30.477 DEBUG iotfunctions.dbtables._store_table_exists Table "BLUADMIN"."KPI_MODEL_STORE" exists.


In [47]:
# need a helper function to convert array columns to something easier
from scipy import linalg
def l2norm(df, tcol, col1, col2 = None, col3 = None):
    def str_norm(cols_str):
        '''norm for one string element (encodes an array of value)
           in one column of a data point'''
        return linalg.norm(np.fromstring(cols_str.replace('[',' ').replace(']','').replace('\"', ''), sep = ','))**2

    def column_norm(df, tcol, col1, col2=None, col3=None):
        '''norm of all columns specified in parameters for all datapoints'''
        df_temp = pd.DataFrame()
        df_temp['col1_np'] = df[col1].apply(str_norm)
        df_temp['col2_np'] = 0 
        df_temp['col3_np'] = 0
        if col2 is not None:
            df_temp['col2_np'] = df[col2].apply(str_norm)
        if col3 is not None:
            df_temp['col3_np'] = df[col3].apply(str_norm)

        return (df_temp['col1_np'] + df_temp['col2_np'] + df_temp['col3_np'])**(1/2)
    
    df[tcol] = column_norm(df, tcol, col1, col2, col3)

def unrollAccel(df):
    l0,l1,l2,l3,l4=[],[],[],[],[]
    for i in df['ACCEL_POWER'].values:
        l0.append(eval(eval(i)[0]))
        l1.append(eval(eval(i)[1]))
        l2.append(eval(eval(i)[2]))
        l3.append(eval(eval(i)[3]))
        l4.append(eval(eval(i)[4]))
    df['accel_power_0'] = np.asarray(l0)
    df['accel_power_1'] = np.asarray(l1)
    df['accel_power_2'] = np.asarray(l2)
    df['accel_power_3'] = np.asarray(l3)
    df['accel_power_4'] = np.asarray(l4)
    
listAttr = ['timestamp','entity','vibrations','rms','accel_speed','accel_power_0','accel_power_1',
            'accel_power_2','accel_power_3','accel_power_4']

In [48]:
# Now we proceed to customer data - GOOD CASE

# Get stuff in
df_input_raw = pd.read_csv('./Armstark04714B6046D5.csv', index_col=False, parse_dates=['RCV_TIMESTAMP_UTC'])
df_input_raw['entity']=df_input_raw['DEVICE_ID']
df_input_raw['timestamp']=df_input_raw['RCV_TIMESTAMP_UTC']

# and sort it by timestamp
df_input_raw = df_input_raw.sort_values(by='timestamp')
df_input_raw = df_input_raw.set_index(['entity','timestamp']).dropna()

l2norm(df_input_raw, 'vibrations', 'VIBRATIONS_XAXIS', 'VIBRATIONS_YAXIS', 'VIBRATIONS_ZAXIS')
l2norm(df_input_raw, 'rms', 'RMS_X', 'RMS_Y', 'RMS_Z')
l2norm(df_input_raw, 'accel_speed', 'ACCEL_SPEED')
unrollAccel(df_input_raw)
#l2norm(df_input_raw, 'accel_power', 'ACCEL_POWER')

df_input = df_input_raw.filter(listAttr, axis=1)
df_input_raw.describe()

,vibrations,rms,accel_speed,accel_power_0,accel_power_1,accel_power_2,accel_power_3,accel_power_4
count,8620.000000,8620.000000,8620.000000,8620.000000,8620.000000,8620.000000,8620.000000,8620.000000
mean,3.759047,2.170460,2208.854495,2.453523,2.453312,2.452808,2.452053,2.452439
std,0.033272,0.002922,589.925540,0.856247,0.857594,0.857678,0.857200,0.856340
min,3.616515,2.155864,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.737696,2.169444,2316.566425,2.251800,2.251800,2.252000,2.252000,2.251800
50%,3.757949,2.170800,2318.802493,2.313800,2.313400,2.311800,2.312200,2.312600
75%,3.780154,2.171949,2379.176328,2.691450,2.692100,2.685100,2.681850,2.682300
max,3.898555,2.184292,2672.101233,5.325799,5.293200,5.334001,5.278600,5.282000


#### Pandas Profiling

Try Pandas Profiling to get an overview about the data, mostly its distributions and correlations
<br>


In [49]:
profile = ProfileReport(df_input, title='Pandas Profiling Report', html={'style':{'full_width':True}})
profile.to_widgets()

2020-03-12T16:37:50.008 DEBUG matplotlib.colorbar._get_ticker_locator_formatter locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x7f4df1d213d0>
2020-03-12T16:37:50.008 DEBUG matplotlib.colorbar.update_ticks Using auto colorbar locator on colorbar
2020-03-12T16:37:50.009 DEBUG matplotlib.colorbar.update_ticks locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x7f4df1d213d0>
2020-03-12T16:37:50.009 DEBUG matplotlib.colorbar._add_solids Setting pcolormesh
2020-03-12T16:37:50.111 DEBUG matplotlib.colorbar._get_ticker_locator_formatter locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x7f4e79913890>
2020-03-12T16:37:50.111 DEBUG matplotlib.colorbar.update_ticks Using auto colorbar locator on colorbar
2020-03-12T16:37:50.112 DEBUG matplotlib.colorbar.update_ticks locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x7f4e79913890>
2020-03-12T16:37:50.112 DEBUG matplotlib.colorbar._add_solids Setting pcolormesh
2020-03-12T16:37:50.212 DEBUG matplotl

In [50]:
df_input2 = df_input.copy()

Indexlist = df_input2.index.to_list()
Indexlist_new = []
for p in Indexlist:
    p_new = ('testentity', p[1])
    Indexlist_new.append(p_new)
df_input2.index = Indexlist_new
df_input = pd.concat([df_input,df_input2])
df_input

vibrations       rms  accel_speed  \
entity       timestamp                                                    
04714B6046D5 2020-01-01 00:02:25.928    3.787826  2.171877  2316.566425   
             2020-01-01 00:07:26.074    3.776503  2.170646  2316.566425   
             2020-01-01 00:12:25.663    3.715573  2.171475  2316.566425   
             2020-01-01 00:17:26.020    3.765538  2.171679  2316.566425   
             2020-01-01 00:22:25.727    3.725934  2.170599  2316.566425   
...                                          ...       ...          ...   
testentity   2020-01-31 11:39:23.476    3.725156  2.171589  2318.802493   
             2020-01-31 11:44:23.497    3.766786  2.170778  2318.802493   
             2020-01-31 11:49:23.443    3.737750  2.171533  2318.802493   
             2020-01-31 11:54:23.499    3.771797  2.172369  2318.802493   
             2020-01-31 11:59:23.501    3.758436  2.171821  2318.802493   

                                      accel_power_0  accel_power_1  \
entity       timestamp                                               
04714B6046D5 2020-01-01 00:02:25.928         2.2904         2.2824   
             2020-01-01 00:07:26.074         2.2822         2.2978   
             2020-01-01 00:12:25.663         2.2718         2.2908   
             2020-01-01 00:17:26.020         2.2748         2.2932   
             2020-01-01 00:22:25.727         2.3320         2.2980   
...                                             ...            ...   
testentity   2020-01-31 11:39:23.476         2.2778         2.2798   
             2020-01-31 11:44:23.497         2.2734         2.2704   
             2020-01-31 11:49:23.443         2.2666         2.2714   
             2020-01-31 11:54:23.499         2.2722         2.2738   
             2020-01-31 11:59:23.501         2.2760         2.2716   

                                      accel_power_2  accel_power_3  \
entity       timestamp                                               
04714B6046D5 2020-01-01 00:02:25.928         2.2858         2.2956   
             2020-01-01 00:07:26.074         2.2966         2.3504   
             2020-01-01 00:12:25.663         2.2912         2.3044   
             2020-01-01 00:17:26.020         2.2926         2.3146   
             2020-01-01 00:22:25.727         2.2998         2.3212   
...                                             ...            ...   
testentity   2020-01-31 11:39:23.476         2.2676         2.2718   
             2020-01-31 11:44:23.497         2.2694         2.2728   
             2020-01-31 11:49:23.443         2.2712         2.2690   
             2020-01-31 11:54:23.499         2.2694         2.2704   
             2020-01-31 11:59:23.501         2.2678         2.2760   

                                      accel_power_4  
entity       timestamp                               
04714B6046D5 2020-01-01 00:02:25.928         2.3278  
             2020-01-01 00:07:26.074         2.3190  
             2020-01-01 00:12:25.663         2.3232  
             2020-01-01 00:17:26.020         2.3606  
             2020-01-01 00:22:25.727         2.2860  
...                                             ...  
testentity   2020-01-31 11:39:23.476         2.2668  
             2020-01-31 11:44:23.497         2.2670  
             2020-01-31 11:49:23.443         2.2646  
             2020-01-31 11:54:23.499         2.2650  
             2020-01-31 11:59:23.501         2.2744  

[17240 rows x 8 columns]

In [51]:
%%time
EngineLogging.configure_console_logging(logging.DEBUG)
agevi = bif.AnomalyGeneratorExtremeValue(input_item=['accel_power_0'], factor=5, size=12, 
                                        output_item=['accel_anomaly'])
                                        
jobsettings = { 'db': db, 
               '_db_schema': 'public', 'save_trace_to_file' : True}
et = agevi._build_entity_type(columns = [Column('accel_power_0',Float()), Column('accel_power_1',Float()),
                                        Column('vibrations',Float())], **jobsettings)
agevi._entity_type = et

df_input = agevi.execute(df=df_input)
df_input.describe()

2020-03-12T16:37:58.112 DEBUG iotfunctions.enginelog.configure_console_logging Console logging has been configured. Level = 10
2020-03-12T16:37:58.113 DEBUG iotfunctions.metadata.__init__ Initializing new entity type using iotfunctions 2.0.3
2020-03-12T16:37:58.114 DEBUG iotfunctions.util.__init__ Starting trace
2020-03-12T16:37:58.114 DEBUG iotfunctions.util.__init__ Trace name: auto_trace_test_entity_for_AnomalyGeneratorExtremeValue_20200312153758
2020-03-12T16:37:58.114 DEBUG iotfunctions.util.__init__ auto_save None
2020-03-12T16:37:58.115 DEBUG iotfunctions.util.categorize_args categorizing arguments
2020-03-12T16:37:58.115 WARNING iotfunctions.metadata.__init__ Created a logical entity type. It is not connected to a real database table, so it cannot perform any database operations.
2020-03-12T16:37:58.138 DEBUG iotfunctions.metadata.__init__ Initialized entity type 
LocalEntityType:TEST_ENTITY_FOR_ANOMALYGENERATOREXTREMEVALUE
Functions:
Granularities:
No schedules metadata
2020-0

In [52]:
df_input[['accel_power_0','accel_anomaly']].head(20)

accel_power_0  accel_anomaly
entity       timestamp                                            
04714B6046D5 2020-01-01 00:02:25.928         2.2904       -27.4848
             2020-01-01 00:07:26.074         2.2822         2.2822
             2020-01-01 00:12:25.663         2.2718         2.2718
             2020-01-01 00:17:26.020         2.2748         2.2748
             2020-01-01 00:22:25.727         2.3320         2.3320
             2020-01-01 00:27:26.057         2.3174       -27.8088
             2020-01-01 00:32:25.866         2.3072         2.3072
             2020-01-01 00:37:26.045         2.3018         2.3018
             2020-01-01 00:42:25.899         2.2868         2.2868
             2020-01-01 00:47:26.049         2.2764         2.2764
             2020-01-01 00:52:26.018         2.3258        27.9096
             2020-01-01 00:57:25.902         2.2950         2.2950
             2020-01-01 01:02:25.974         2.2832         2.2832
             2020-01-01 01:07:26.172         2.2836         2.2836
             2020-01-01 01:12:26.076         2.2826         2.2826
             2020-01-01 01:17:26.169         2.2864       -27.4368
             2020-01-01 01:22:26.149         2.2760         2.2760
             2020-01-01 01:27:26.106         2.2854         2.2854
             2020-01-01 01:32:26.242         2.2828         2.2828
             2020-01-01 01:37:26.190         2.2628         2.2628

#### Customer suggested a correlation between vibration and acceleration

so let's try to predict (although correlation tests do not really indicate it)

In [53]:
# Run Monitoring's anomaly detector functions
EngineLogging.configure_console_logging(logging.DEBUG)

gbmii = GBMRegressor(features=['accel_power_0','accel_power_1'], max_depth=20, num_leaves=40,
                     n_estimators=4000, learning_rate=0.00001, targets=['rms'],
                     predictions=['rms_pred'])
gbmii.correlation_threshold = 0.001

jobsettings = { 'db': db, 
               '_db_schema': 'public', 'save_trace_to_file' : True}
et = gbmii._build_entity_type(columns = [Column('accel_power_0',Float()), Column('accel_power_1',Float()),
                                        Column('vibrations',Float())], **jobsettings)
gbmii._entity_type = et

# allow training and delete existing models
gbmii.auto_train = True
gbmii.delete_existing_models = True
df_input = gbmii.execute(df=df_input)

2020-03-12T16:38:01.446 DEBUG iotfunctions.enginelog.configure_console_logging Console logging has been configured. Level = 10
2020-03-12T16:38:01.447 DEBUG iotfunctions.metadata.__init__ Initializing new entity type using iotfunctions 2.0.3
2020-03-12T16:38:01.448 DEBUG iotfunctions.util.__init__ Starting trace
2020-03-12T16:38:01.448 DEBUG iotfunctions.util.__init__ Trace name: auto_trace_test_entity_for_GBMRegressor_20200312153801
2020-03-12T16:38:01.449 DEBUG iotfunctions.util.__init__ auto_save None
2020-03-12T16:38:01.449 DEBUG iotfunctions.util.categorize_args categorizing arguments
2020-03-12T16:38:01.449 WARNING iotfunctions.metadata.__init__ Created a logical entity type. It is not connected to a real database table, so it cannot perform any database operations.
2020-03-12T16:38:01.450 DEBUG iotfunctions.metadata.__init__ Initialized entity type 
LocalEntityType:TEST_ENTITY_FOR_GBMREGRESSOR
Functions:
Granularities:
No schedules metadata
2020-03-12T16:38:01.451 DEBUG mmfuncti

2020-03-12T16:38:38.788 INFO iotfunctions.base.get_models_for_training predicting target rms
2020-03-12T16:38:39.622 INFO iotfunctions.dbtables.retrieve_model Model model.TEST_ENTITY_FOR_GBMREGRESSOR.GBMRegressor.rms.testentity does not exist in table "BLUADMIN"."KPI_MODEL_STORE"
2020-03-12T16:38:39.623 INFO iotfunctions.base.get_models_for_training load model None
2020-03-12T16:38:39.624 INFO iotfunctions.base.get_models_for_training training required: True  results: Training required because there is no existing model
2020-03-12T16:38:39.625 DEBUG iotfunctions.pipeline.execute Executing pipeline with 0 stages.
2020-03-12T16:38:39.625 DEBUG iotfunctions.pipeline.execute columns excluded when dropping null rows ['deviceid', '_timestamp', 'logicalinterface_id', 'devicetype', 'format', 'updated_utc', 'evt_timestamp']
2020-03-12T16:38:39.626 DEBUG iotfunctions.pipeline.execute columns considered when dropping null rows ['vibrations', 'rms', 'accel_speed', 'accel_power_0', 'accel_power_1',

In [11]:
df_input

vibrations       rms  accel_speed  \
entity       timestamp                                                    
04714B6046D5 2020-01-01 00:02:25.928    3.787826  2.171877  2316.566425   
             2020-01-01 00:07:26.074    3.776503  2.170646  2316.566425   
             2020-01-01 00:12:25.663    3.715573  2.171475  2316.566425   
             2020-01-01 00:17:26.020    3.765538  2.171679  2316.566425   
             2020-01-01 00:22:25.727    3.725934  2.170599  2316.566425   
...                                          ...       ...          ...   
testentity   2020-01-31 11:39:23.476    3.725156  2.171589  2318.802493   
             2020-01-31 11:44:23.497    3.766786  2.170778  2318.802493   
             2020-01-31 11:49:23.443    3.737750  2.171533  2318.802493   
             2020-01-31 11:54:23.499    3.771797  2.172369  2318.802493   
             2020-01-31 11:59:23.501    3.758436  2.171821  2318.802493   

                                      accel_power_0  accel_power_1  \
entity       timestamp                                               
04714B6046D5 2020-01-01 00:02:25.928         2.2904         2.2824   
             2020-01-01 00:07:26.074         2.2822         2.2978   
             2020-01-01 00:12:25.663         2.2718         2.2908   
             2020-01-01 00:17:26.020         2.2748         2.2932   
             2020-01-01 00:22:25.727         2.3320         2.2980   
...                                             ...            ...   
testentity   2020-01-31 11:39:23.476         2.2778         2.2798   
             2020-01-31 11:44:23.497         2.2734         2.2704   
             2020-01-31 11:49:23.443         2.2666         2.2714   
             2020-01-31 11:54:23.499         2.2722         2.2738   
             2020-01-31 11:59:23.501         2.2760         2.2716   

                                      accel_power_2  accel_power_3  \
entity       timestamp                                               
04714B6046D5 2020-01-01 00:02:25.928         2.2858         2.2956   
             2020-01-01 00:07:26.074         2.2966         2.3504   
             2020-01-01 00:12:25.663         2.2912         2.3044   
             2020-01-01 00:17:26.020         2.2926         2.3146   
             2020-01-01 00:22:25.727         2.2998         2.3212   
...                                             ...            ...   
testentity   2020-01-31 11:39:23.476         2.2676         2.2718   
             2020-01-31 11:44:23.497         2.2694         2.2728   
             2020-01-31 11:49:23.443         2.2712         2.2690   
             2020-01-31 11:54:23.499         2.2694         2.2704   
             2020-01-31 11:59:23.501         2.2678         2.2760   

                                      accel_power_4 vibration_pred  
entity       timestamp                                              
04714B6046D5 2020-01-01 00:02:25.928         2.3278        3.75934  
             2020-01-01 00:07:26.074         2.3190        3.75899  
             2020-01-01 00:12:25.663         2.3232        3.75924  
             2020-01-01 00:17:26.020         2.3606        3.75924  
             2020-01-01 00:22:25.727         2.2860        3.75934  
...                                             ...            ...  
testentity   2020-01-31 11:39:23.476         2.2668        3.75905  
             2020-01-31 11:44:23.497         2.2670        3.75886  
             2020-01-31 11:49:23.443         2.2646        3.75899  
             2020-01-31 11:54:23.499         2.2650        3.75946  
             2020-01-31 11:59:23.501         2.2744        3.75886  

[34480 rows x 9 columns]

In [13]:
print (gbmii._entity_type.db.model_store)

In [2]:
# now predict 
EngineLogging.configure_console_logging(logging.DEBUG)
gbmii = GBMRegressor(features=['accel_power_0'], max_depth=100000, num_leaves=40, threshold=2,
                     n_estimators=1000, learning_rate=0.001, targets=['vibrations'],
                     predictions=['vibration_pred'])

jobsettings = { 'db': db, 
               '_db_schema': 'public', 'save_trace_to_file' : True}
et = gbmii._build_entity_type(columns = [Column('accel_power0',Float())], **jobsettings)
gbmii._entity_type = et

df_input = gbmii.execute(df=df_input)

NameError: name 'EngineLogging' is not defined